In [4]:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
import io
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
from pyathena import connect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, \
RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_curve, auc, mean_squared_error,\
precision_score, recall_score, f1_score, accuracy_score,\
confusion_matrix, plot_confusion_matrix, classification_report
from sagemaker.tuner import HyperparameterTuner
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import loguniform
import warnings
warnings.filterwarnings('ignore')
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [5]:
ingest_create_athena_db_passed = False

In [6]:
database_name = "ecommerce"

In [7]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [8]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [9]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS ecommerce


""


In [10]:
#S3 bucket name: olistteam5
#S3 URL: s3://olistteam5/ecommerce/

In [85]:
olist_dir='s3://olistteam5/ecommerce'

In [87]:
table_name ='olist_order_items_dataset'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


ccreate_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                order_item_id INT,
                product_id STRING,
                seller_id STRING,
                shipping_limit_date STRING,
                price FLOAT,
                freight_value FLOAT
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{olist_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""
pd.read_sql(create_table, conn)
pd.read_sql(f'SELECT * FROM ecommerce.olist_order_items_dataset LIMIT 10', conn)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,"""00010242fe8c5a6d1ba2dd792cb16214""",1,"""4244733e06e7ecb4970a6e2683c13e61""","""48436dade18ac8b2bce089ec2a041202""",2017-09-19 09:45:35,58.90,13.29
1,"""00018f77f2f0320c557190d7a144bdd3""",1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,"""000229ec398224ef6ca0657da4fc703e""",1,c777355d18b72b67abbeef9df44fd0fd,"""5b51032eddd242adc84c38acab88f23d""",2018-01-18 14:48:30,199.00,17.87
3,"""00024acbcdf0a6daa1e931b038114c75""",1,"""7634da152a4610f1595efa32f14722fc""","""9d7a1d34a5052409006425275ba1c2b4""",2018-08-15 10:10:18,12.99,12.79
4,"""00042b26cf59d7ce69dfabb4e55b4fd9""",1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
5,"""00048cc3ae777c65dbb7d2a0634bc1ea""",1,ef92defde845ab8450f9d70c526ef70f,"""6426d21aca402a131fc0a5d0960a3c90""",2017-05-23 03:55:27,21.90,12.69
6,"""00054e8431b9d7675808bcb819fb4a32""",1,"""8d4f2bb7e93e6710a28f34fa83ee7d28""","""7040e82f899a04d1b434b795a43b4617""",2017-12-14 12:10:31,19.90,11.85
7,"""000576fe39319847cbb9d288c5617fa6""",1,"""557d850972a7d6f792fd18ae1400d9b6""","""5996cddab893a4652a15592fb58ab8db""",2018-07-10 12:30:45,810.00,70.75
8,"""0005a1a1728c9d785b8e2b08b904576c""",1,"""310ae3c140ff94b03219ad0adc3c778f""",a416b6a846a11724393025641d4edd5e,2018-03-26 18:31:29,145.95,11.65
9,"""0005f50442cb953dcd1d21e1fb923495""",1,"""4535b0e1091c278dfd193e5a1d63b39f""",ba143b05f0110f0dc71ad71b4466ce92,2018-07-06 14:10:56,53.99,11.40
